In [1]:
import pandas as pd
import zipfile
import glob
import os

In [2]:
def createDictionary(dictionary,data):
#     data = pd.read_table(path,sep='\t')
    #initialize distribution dataframe
    distribution = pd.DataFrame()
    distribution['path'] = data.iloc[:,0]
    
    #drop path
    data = data.drop("path", axis=1)
    #find theme from the curator indictaor
    curated = data.filter(regex='.ind$',axis=1)
    curated['max'] = curated.idxmax(axis = 1, skipna = True)
    curated['maxValue'] = curated.max(axis=1)
    print(curated.head())
    #find theme from highest matching predicted value
    predicted = data[data.columns.drop(list(data.filter(regex='.ind$')))]
    predicted['max'] = predicted.idxmax(axis = 1, skipna = True)
    predicted['maxValue'] = predicted.max(axis=1)
    print(predicted.head())
    
    #update the theme with curated value
    distribution['theme'] = curated.apply(select_curated, axis=1)
    #if value is set to -1 set with predicted value
    rows = (distribution[['theme']] == -1).all(axis=1)
    distribution.loc[rows,'theme'] = predicted.loc[rows,'max']
    
    #convert distribution to dictionary
    dist_dict= dict([(i,a) for i, a in zip(distribution.path,distribution.theme )])
    dictionary.update(dist_dict)

In [3]:
# helper function for setting the curated theme
def select_curated(row):
    if row['maxValue']  > 0:
        val = row['max']
    else:
        val = -1
    return val

In [4]:
def add_theme(dependency_path):
    return dictionary.get(dependency_path.lower())

In [5]:
filelist = os.listdir(os.getcwd())
# initialize dictionary for files(i)
dictionary = {}
#initialize the dataframe for (ii)
dataframe_list = []
# import zipfile
for filename in filelist:
    if zipfile.is_zipfile(filename):
        zf = zipfile.ZipFile(filename, 'r')
        flist = zf.namelist()
        flist_splitname = flist[0].split('-')
        if flist_splitname[1] == 'i':
            #create the dictionary from file i
            theme_distribution = pd.read_table(zf.open(flist[0]),sep='\t')
            createDictionary(dictionary,theme_distribution)
            print(flist_splitname)
        elif flist_splitname[1] == 'ii':            
            # map the path with corresponding theme in part ii
            header = ['pubmed_id','sentence_id','mention1_offset','mention2_offset','mention1','mention2','geneids1','geneids2','type1','type2', 'dependency_path']
            data_with_theme = pd.read_csv(zf.open(flist[0]),
                              sep='\t',
                              names=header,
                              usecols=[0,1,3,5,2,4,8,9,10,11,12])
            print(len(data_with_theme))
            data_with_theme['theme'] = data_with_theme.dependency_path.apply(add_theme)
            data_with_theme = data_with_theme.dropna()
            print(len(data_with_theme))
            dataframe_list.append(data_with_theme)
            print(len(dataframe_list))
all_data = pd.concat(dataframe_list)
print(len(all_data))

C:\Users\nayan\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\nayan\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


   T.ind  C.ind  Sa.ind  Pr.ind  Pa.ind  J.ind  Mp.ind    max  maxValue
0      0      0       0       0       0      0       0  T.ind         0
1      0      0       0       0       0      0       0  T.ind         0
2      0      0       0       0       0      0       0  T.ind         0
3      0      0       0       0       0      0       0  T.ind         0
4      0      0       0       0       0      0       0  T.ind         0


C:\Users\nayan\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\nayan\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


     T    C   Sa   Pr   Pa    J   Mp max  maxValue
0  1.0  0.0  0.0  0.0  0.0  0.0  0.0   T       1.0
1  0.0  0.0  0.0  0.0  0.0  1.0  0.0   J       1.0
2  0.0  0.0  0.0  0.0  1.0  1.0  0.0  Pa       1.0
3  0.0  0.0  0.0  0.0  1.0  0.0  0.0  Pa       1.0
4  1.0  0.0  0.0  0.0  0.0  0.0  0.0   T       1.0
['part', 'i', 'chemical', 'disease', 'path', 'theme', 'distributions.txt']
   A+.ind  A-.ind  B.ind  E+.ind  E-.ind  E.ind  N.ind  O.ind  K.ind  Z.ind  \
0       0       0      0       0       0      0      0      0      0      0   
1       0       0      0       0       0      0      0      0      0      0   
2       0       0      0       0       0      0      0      0      0      0   
3       0       0      0       0       0      0      0      0      0      0   
4       0       0      0       0       0      0      0      0      0      0   

      max  maxValue  
0  A+.ind         0  
1  A+.ind         0  
2  A+.ind         0  
3  A+.ind         0  
4  A+.ind         0  
    A+   A- 

In [7]:
#write to file
# file = open('data_with_theme.txt','w',encoding='utf-8')
# file.write(str(all_data))
# file.close()
all_data.to_csv('all_data_with_theme.txt')